In [1]:
import pandas as pd
import numpy as np
import dgl.nn as dglnn
from dgl import from_networkx
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
from dgl.data.utils import load_graphs
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler
from sklearn.manifold import TSNE

In [3]:
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, e_dim):
        super(GATLayer, self).__init__()
        self.w1 = nn.Linear(in_dim + e_dim, out_dim, bias=False)
        self.w2 = nn.Linear(in_dim + out_dim, out_dim, bias=False)
        self.w_att = nn.Linear(in_dim + out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.w1.weight, gain=gain)
        nn.init.xavier_normal_(self.w2.weight, gain=gain)
        nn.init.xavier_normal_(self.w_att.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['h'], edges.data['m']], -1)
        a = self.w_att(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def msg1(self, edges):
        return {'m': self.w1(th.cat([edges.src['h'], edges.data['h']], -1))}
    
    def message_func(self, edges):
        return {'z': edges.data['m'], 'e': edges.data['e']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h_neigh': h}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = hfeat
            g.edata['h'] = efeat
            g.apply_edges(self.msg1)
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            g.ndata['h'] = F.relu(self.w2(th.cat([g.ndata['h'], g.ndata['h_neigh']], -1)))
            return g.ndata['h']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, e_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim, e_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim, e_dim)
        self.pred = MLPPredictor(out_dim, 5)
    def forward(self, g, nfeats, efeats):
        nfeats = self.layer1(g, nfeats, efeats)
        nfeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [4]:
G = load_graphs("./botlot/All features/bot_data_rus.bin")[0][0]

In [5]:
G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1]) 
G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype= th.bool)

In [6]:
G

Graph(num_nodes=261186, num_edges=514292,
      ndata_schemes={'h': Scheme(shape=(52,), dtype=torch.float32)}
      edata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'h': Scheme(shape=(52,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64)})

In [7]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(G.edata['label'].cpu().numpy()),
                                                 G.edata['label'].cpu().numpy())
class_weights = th.FloatTensor(class_weights)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4], y=[1 1 1 ... 1 1 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [8]:
device = 'cuda:3'

In [9]:
G = G.to(device)

In [10]:
class_weights = class_weights.to(device)

In [11]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata['label']
train_mask = G.edata['train_mask']

In [12]:
in_dim = 52
hidden_dim = 20
out_dim = 10
e_dim = 52

In [13]:
model = Model(in_dim, hidden_dim, out_dim, e_dim)

In [14]:
opt = th.optim.Adam(model.parameters())

In [15]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [16]:
import time

In [17]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [35]:
model = model.to(device)
for epoch in range(9300,15000):
    pred = model(G, node_features, edge_features)
    loss = criterion(pred[train_mask], edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
        print('loss',loss.item(),'Epoch:', epoch ,' Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))
    if epoch % 100 == 0:
        torch.save(model.state_dict(), 'egat_bot_model' + str(epoch))

loss 0.0003645187243819237 Epoch: 9300  Training acc: 0.9999319911003113
loss 0.0003376924723852426 Epoch: 9400  Training acc: 0.9999339580535889
loss 0.0003129641991108656 Epoch: 9500  Training acc: 0.9999358654022217
loss 0.00029040133813396096 Epoch: 9600  Training acc: 0.9999397993087769
loss 0.00026952457847073674 Epoch: 9700  Training acc: 0.999957263469696
loss 0.0002501259441487491 Epoch: 9800  Training acc: 0.999957263469696
loss 0.00023205921752378345 Epoch: 9900  Training acc: 0.999963104724884
loss 0.00021521009330172092 Epoch: 10000  Training acc: 0.999963104724884
loss 0.0001994153280975297 Epoch: 10100  Training acc: 0.9999650716781616
loss 0.0001845482038334012 Epoch: 10200  Training acc: 0.999968945980072
loss 0.0001710810756776482 Epoch: 10300  Training acc: 0.999968945980072
loss 0.0001586510188644752 Epoch: 10400  Training acc: 0.999968945980072
loss 0.0001464634551666677 Epoch: 10500  Training acc: 0.9999708533287048
loss 0.00013521304936148226 Epoch: 10600  Traini

In [36]:
model = model.to('cpu')
cm, cr = test(G_test, model)
print(cm)

[[115594      4      0      0      0]
 [     4  99012      0      0      0]
 [     2      0    279      5      0]
 [     0      0      8   5454      2]
 [     0      0      0      0     48]]


In [37]:
print(cr)

              precision    recall  f1-score   support

           0     0.9999    1.0000    1.0000    115598
           1     1.0000    1.0000    1.0000     99016
           2     0.9721    0.9755    0.9738       286
           3     0.9991    0.9982    0.9986      5464
           4     0.9600    1.0000    0.9796        48

    accuracy                         0.9999    220412
   macro avg     0.9862    0.9947    0.9904    220412
weighted avg     0.9999    0.9999    0.9999    220412



In [29]:
def test(G_test, model):
    test_node_features = G_test.ndata['h']
    test_edge_features = G_test.edata['h']
    y_true = G_test.edata['label'].detach().numpy()
    pred = model(G_test, test_node_features, test_edge_features)
    y_pred = pred.detach().numpy()
    y_pred = np.argmax(y_pred, -1)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, digits=4)
    return cm, cr

In [21]:
G_test = load_graphs("./botlot/All features/bot_test_data_rus.bin")[0][0]

In [22]:
G_test.ndata['h'] = th.ones(G_test.num_nodes(), G_test.edata['h'].shape[1]) 

In [23]:
Counter(G_test.edata['label'].numpy())

Counter({1: 99016, 0: 115598, 3: 5464, 2: 286, 4: 48})

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, in_dim , out_dim):
        super(GATLayer, self).__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(3 * out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.W.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['z'], edges.dst['z'], edges.data['z']], -1)
        a = self.attn_fc(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e'] ,'m': edges.data['z']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['m'], dim=1)
        return {'h': F.relu(h)}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            z1 = self.W(hfeat)
            z2 = self.W(efeat)
            g.ndata['z'] = z1 # 每个节点的特征
            g.edata['z'] = z2 # 每条边的特征
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            return g.ndata['h'], g.edata['z']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim)
        self.pred = MLPPredictor(out_dim, 5)
    def forward(self, g, nfeats, efeats):
        nfeats, efeats = self.layer1(g, nfeats, efeats)
#         nfeats, efeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']